### Making connection to database

In [1]:
from mysql_connection import MySqlManip
from neo4j_connection import Neo4jManip
import pandas as pd

conn = MySqlManip(
    user='root', host='10.60.5.99', port=3306, password='Boway@123', database='IOT'
)

# conn.create_params_table()

### Reading historian data

In [5]:
data = conn.load_data_to_dataframe('device_4_bubble_cleaned_data_bayes')
data = data.dropna()
selected_labels = ["cool_strength_total_mean_3min", "temp_jiaozhu", "label"]
data = data[selected_labels]

data["cool_strength_total_mean_3min"] = pd.cut(
    data["cool_strength_total_mean_3min"],
    [-float('inf'), 2400, 2600, 2700, 2900, float('inf')],
    labels=["Lower_than_2400", "2400-2600", "2600-2700", "2700-2900", "Greater_than_2900"])

data["temp_jiaozhu"] = pd.cut(
    data["temp_jiaozhu"],
    [-float('inf'), 1230, 1235, 1245, 1250, float('inf')],
    labels=["Lower_than_1230","1230-1235","1235-1245", "1245-1250", "Greater_than_1250"])

data["label"] = pd.cut(
    data["label"],
    2,
    labels=["FineBatch","BadBatch"])

data

,cool_strength_total_mean_3min,temp_jiaozhu,label
0,Lower_than_2400,1235-1245,FineBatch
1,Lower_than_2400,1235-1245,FineBatch
2,Lower_than_2400,1235-1245,FineBatch
3,Lower_than_2400,1235-1245,FineBatch
4,Lower_than_2400,1235-1245,FineBatch
...,...,...,...
16208,Lower_than_2400,1235-1245,FineBatch
16209,Lower_than_2400,1235-1245,FineBatch
16210,Lower_than_2400,1235-1245,FineBatch
16211,Lower_than_2400,1235-1245,FineBatch


### Creating BN model

In [9]:
# from models.bayes_net import BNCreator
# bn = BNCreator(["cool_strength_total_mean_3min", "temp_jiaozhu"], ['label'])

from pgmpy.models import BayesianNetwork
model = BayesianNetwork([
    ("cool_strength_total_mean_3min", "D"),
    ("temp_jiaozhu", "D"),
    ("D", 'label'),
], latents={"D"})

Fitting model with data (initialize model)

In [11]:
# bn.model.fit(data)
# bn.model.get_cpds()[0].state_names

from pgmpy.estimators import ExpectationMaximization
estimator = ExpectationMaximization(model, data)

params = estimator.get_parameters(latent_card={"D": 3})
model.add_cpds(*params)


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:03<00:00, 27.77it/s]


In [12]:
# print(bn.model.get_cpds()[0])

print(model.get_cpds())


[<TabularCPD representing P(temp_jiaozhu:4) at 0x1f7b9c9df40>, <TabularCPD representing P(cool_strength_total_mean_3min:5) at 0x1f7ba23c5e0>, <TabularCPD representing P(label:2 | D:3) at 0x1f7b9d24a30>, <TabularCPD representing P(D:3 | cool_strength_total_mean_3min:5, temp_jiaozhu:4) at 0x1f7ba294520>]


Making changes towards cpd (prior probability)

In [7]:
bn.update_cpd(
    "cool_strength_total_mean_3min",
    {
        ""
        "2400-2600": 0.08,
        "OverCooled": 0.13,
        "WellCooled": 0.79
    }
)

In [8]:
print(bn.model.get_cpds()[0])

+-------------------------------------------+------+
| cool_strength_total_mean_3min(2400-2600)  | 0.08 |
+-------------------------------------------+------+
| cool_strength_total_mean_3min(OverCooled) | 0.13 |
+-------------------------------------------+------+
| cool_strength_total_mean_3min(WellCooled) | 0.79 |
+-------------------------------------------+------+


In [9]:
print(bn.model.get_cpds()[1])

+-------------------------------+-----+------------------------------------------------+
| cool_strength_total_mean_3min | ... | cool_strength_total_mean_3min(Lower_than_2400) |
+-------------------------------+-----+------------------------------------------------+
| temp_jiaozhu                  | ... | temp_jiaozhu(Lower_than_1230)                  |
+-------------------------------+-----+------------------------------------------------+
| label(BadBatch)               | ... | 0.14878892733564014                            |
+-------------------------------+-----+------------------------------------------------+
| label(FineBatch)              | ... | 0.8512110726643599                             |
+-------------------------------+-----+------------------------------------------------+


In [10]:
print(bn.model.get_cpds()[2])

+-------------------------------+------------+
| temp_jiaozhu(1230-1235)       | 0.0150497  |
+-------------------------------+------------+
| temp_jiaozhu(1235-1245)       | 0.956023   |
+-------------------------------+------------+
| temp_jiaozhu(1245-1250)       | 0.00882008 |
+-------------------------------+------------+
| temp_jiaozhu(Lower_than_1230) | 0.0201073  |
+-------------------------------+------------+


### Inference

In [14]:
sample_data = data[["cool_strength_total_mean_3min", "temp_jiaozhu"]].loc[[10300]]

demon_data = pd.DataFrame( sample_data)
demon_data.columns = ["冷却强度", "浇铸温度"]
demon_data

,冷却强度,浇铸温度
10300,Lower_than_2400,1235-1245


In [15]:
res = model.predict_probability(sample_data)

# res.columns = ["有气泡批次", "无气泡批次"]
res

,label_BadBatch,label_FineBatch,D_0,D_1,D_2
10300,0.08516,0.91484,0.599703,0.331146,0.069151


In [34]:
sample_data

,label
10300,BadBatch
